In [194]:
#imports to work with...
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
from torch.utils.data import DataLoader
import torch
import torchvision
from torchvision import transforms
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import label_binarize

from cl_framework.continual_learning.metrics.metric_evaluator_incdec import MetricEvaluatorIncDec
from cl_framework.utilities.matrix_logger import IncDecLogger
from torchmetrics import Recall

In [195]:
results_path = '../runs_trainings/no_freeze/incremental_decremental_multiclass/normal'
seeds = [0,1,2]
statistics_save_path = os.path.join(results_path,'statistics')
if not os.path.exists(statistics_save_path):
    os.mkdir(statistics_save_path)
mean_std_save_path = os.path.join(statistics_save_path,'mean_std_from_seeds')
if not os.path.exists(mean_std_save_path):
    os.mkdir(mean_std_save_path)
mean_save_path = os.path.join(statistics_save_path,'mean_from_seeds')
if not os.path.exists(mean_save_path):
    os.mkdir(mean_save_path)
std_save_path = os.path.join(statistics_save_path,'std_from_seeds')
if not os.path.exists(std_save_path):
    os.mkdir(std_save_path)

In [196]:
scalars_file_names = ['acc.out', 'forg_acc.out','forg_mean_ap.out','map_weighted.out','mean_ap.out']
multidim_file_names = ['acc_per_class.out','ap.out','forg_accuracy_per_subcategory.out',
                       'forg_ap_per_subcategory.out','forg_ap.out','forg_recall_per_subcategory.out',
                       'precision_per_class.out','recall_per_class.out']

In [197]:
data = {}
for file_name in scalars_file_names:
    data[file_name] = []
    for idx_seed in seeds:
        seed_path = os.path.join(results_path, 'seed_' + str(idx_seed))
        
        for exp_dir in os.listdir(seed_path):
            exp_path = os.path.join(seed_path,exp_dir)
            logger_path = os.path.join(exp_path,'logger')
            file_path = os.path.join(logger_path,file_name)
            file_data = np.loadtxt(file_path,delimiter=',')
            data[file_name].append(file_data)

In [198]:
data_mean = {}
data_std = {}
data_string = {}
for file_name in scalars_file_names:
    data_mean[file_name] = np.mean(data[file_name], axis=0)
    data_std[file_name] = np.std(data[file_name], axis=0, ddof=1)

    
    np.savetxt(os.path.join(mean_save_path,file_name), data_mean[file_name],delimiter=',',fmt='%.3f')
    np.savetxt(os.path.join(std_save_path,file_name), data_std[file_name],delimiter=',',fmt='%.3f')
    
    tmp_mean = data_mean[file_name].tolist()
    tmp_std = data_std[file_name].tolist()
    tmp_string_list = []
    for idx in range(len(tmp_mean)):
        tmp_string = "{:.3f}".format(tmp_mean[idx])+'\u00B1'+"{:.2f}".format(tmp_std[idx])
        tmp_string_list.append(tmp_string)
    data_string[file_name] = tmp_string_list
    np.savetxt(os.path.join(mean_std_save_path,file_name), data_string[file_name],delimiter=',',fmt='%s')



In [199]:
data = {}
for file_name in multidim_file_names:
    data[file_name] = []
    for idx_seed in seeds:
        seed_path = os.path.join(results_path, 'seed_' + str(idx_seed))
        
        for exp_dir in os.listdir(seed_path):
            exp_path = os.path.join(seed_path,exp_dir)
            logger_path = os.path.join(exp_path,'logger')
            file_path = os.path.join(logger_path,file_name)
            file_data = np.loadtxt(file_path,delimiter=',')
            data[file_name].append(file_data)

In [200]:
data_mean = {}
data_std = {}
data_string = {}
for file_name in multidim_file_names:
    data_mean[file_name] = np.mean(data[file_name], axis=0)
    data_std[file_name] = np.std(data[file_name], axis=0, ddof=1)

    np.savetxt(os.path.join(mean_save_path,file_name), data_mean[file_name],delimiter=',',fmt='%.3f')
    np.savetxt(os.path.join(std_save_path,file_name), data_std[file_name],delimiter=',',fmt='%.3f')
    
    tmp_mean = data_mean[file_name].tolist()
    tmp_std = data_std[file_name].tolist()
    tmp_string_array = []
    for i in range(len(tmp_mean)):
        tmp_string_list = []
        for j in range(len(tmp_mean[i])):
            tmp_string = "{:.3f}".format(tmp_mean[i][j])+'\u00B1'+"{:.2f}".format(tmp_std[i][j])
            tmp_string_list.append(tmp_string)
        tmp_string_array.append(tmp_string_list)
        data_string[file_name] = tmp_string_array
        np.savetxt(os.path.join(mean_std_save_path,file_name), data_string[file_name],delimiter=',',fmt='%s')